## SPARC-4 mini-pipeline tools
# Stack science images

This notebook shows an example for reducing a series of scientific images using the SPARC4 pipeline to produce a master stacked image of a given field.

* Eder Martioli, LNA 10 Jun 2023

In [1]:
import os,sys
path_parent = os.path.dirname(os.getcwd())
# path_parent = "/Volumes/Samsung_T5/sparc4-pipeline/"
sys.path.append(path_parent) 
import sparc4_pipeline_lib as s4pipelib
import sparc4_product_plots as s4plt

/Users/eder/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


## User inputs

In [3]:
# set an object ID matching the ID in the image header keyword 'OBJECT'
OBJECTID = "CR1"

# set night directory name
NIGHTDIR = '20230503'
# set raw data root directory
DATADIR =  "/Volumes/Samsung_T5/Data/SPARC4/minidata/"
# set reduced data root directory
REDUCEDDIR = "/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/"

# set SPARC4 channel
CHANNEL = 1  # 1, 2, 3 or 4

# whether or not to force reduction even if product already exists
FORCE = False

# get SPARC4 pipeline parameters
p = s4pipelib.init_s4_p(datadir=DATADIR,
                        reducedir=REDUCEDDIR,
                        nightdir=NIGHTDIR,
                        channels="{}".format(CHANNEL),
                        print_report=False)

# set maximum number of files for stacking
p['NFILES_FOR_STACK'] = 3

# set S/N threshold for detecting sources in stack
p['PHOT_THRESHOLD'] = 10

## Define input calibrations
Here is where one should define the following calibrations:
* Master bias
* Master flat
* A file containing the reference WCS for an astrometric solution

In [4]:
# set master bias product file name
p["master_bias"] = "{}/{}_s4c{}_MasterZero.fits".format(p['reduce_directories'][CHANNEL-1],NIGHTDIR,CHANNEL)
# set master flat product file name
p["master_flat"] = "{}/{}_s4c{}_MasterDomeFlat.fits".format(p['reduce_directories'][CHANNEL-1],NIGHTDIR,CHANNEL)

# set reference image for rough astrometry 
p["ASTROM_REF_IMG"] = p["ASTROM_REF_IMGS"][CHANNEL-1]

## 1. To produce an `ASTROMETRY` reference

In [5]:
# input file list for stack
sci_list = p['objsInPhotdata'][CHANNEL-1][OBJECTID.replace(" ","")]

# directory path to save astrometry reference
reduce_dir = p['reduce_directories'][CHANNEL-1]

# set suffix for the stack image name
stack_suffix = "{}_s4c{}_{}_astrometryRef".format(NIGHTDIR,p['CHANNELS'][CHANNEL-1],OBJECTID.replace(" ",""))

# calculate stack
p = s4pipelib.stack_science_images(p,
                                   sci_list,
                                   reduce_dir=reduce_dir,
                                   force=True,
                                   stack_suffix=stack_suffix,
                                   polarimetry=False)

4 Reference image: /Volumes/Samsung_T5/Data/SPARC4/minidata//sparc4acs1/20230503/20230503_s4c1_002026_cr1.fits
OBJECT files: 3
Loading science frames to memory ... 
gain: 0.8 electron / adu
Calibrating science frames (CR, gain, bias, flat) ... 
Calibrating science frame 1 of 3 : 20230503_s4c1_002026_cr1.fits 


/Users/eder/anaconda3/lib/python3.7/site-packages/astropop/_db.py:123: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if k in self.keywords or add_columns:
/Users/eder/anaconda3/lib/python3.7/site-packages/astropop/_db.py:82: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.keywords:
/Users/eder/anaconda3/lib/python3.7/site-packages/astropop/_db.py:63: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if name in self.keywords:
/Users/eder/anaconda3/lib/python3.7/site-packages/astropop/_db.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  while col in self.keywords:


Calibrating science frame 2 of 3 : 20230503_s4c1_002025_cr1.fits 
Calibrating science frame 3 of 3 : 20230503_s4c1_002024_cr1.fits 


2023-06-11 04:47:32,739 astropop - INFO - Registering image 1 from 3  [register]
2023-06-11 04:47:32,749 astropop - INFO - Images are equal, skipping registering.  [register]
2023-06-11 04:47:32,830 astropop - INFO - Registering image 2 from 3  [register]


Registering science frames and stacking them ... 
0 /Volumes/Samsung_T5/Data/SPARC4/minidata//sparc4acs1/20230503/20230503_s4c1_002026_cr1.fits
1 /Volumes/Samsung_T5/Data/SPARC4/minidata//sparc4acs1/20230503/20230503_s4c1_002025_cr1.fits
2 /Volumes/Samsung_T5/Data/SPARC4/minidata//sparc4acs1/20230503/20230503_s4c1_002024_cr1.fits


2023-06-11 04:47:33,120 astropop - INFO - Filling registered image with cval=24.90  [register]
2023-06-11 04:47:33,120 astropop - INFO - Registering image with: translation=[-0.04100476 -0.3047347 ], rotation=0.00°  [register]
2023-06-11 04:47:33,195 astropop - INFO - Registering image 3 from 3  [register]
2023-06-11 04:47:33,463 astropop - INFO - Filling registered image with cval=24.09  [register]
2023-06-11 04:47:33,464 astropop - INFO - Registering image with: translation=[-0.02307641  0.214458  ], rotation=0.00°  [register]
2023-06-11 04:47:33,687 astropop - INFO - Combining 3 images with median method.  [imcombine]


Creating new catalog of detected sources:


In [ ]:
s4plt.plot_sci_frame(p['OBJECT_STACK'], nstars=20, use_sky_coords=True)

In [6]:
p['OBJECT_STACK']

'/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced//sparc4acs1//20230503/20230503_s4c1_CR1_astrometryRef_stack.fits'